# Convert to Full Vertex skymap

## Overview

In this notebook we will:
- walk through the convertion of an lsst skymap file into a "Full Vertex" skymap file
- a Full Vertex file is compoed of the vertices of all the tracts in the skymap, represented in radec, and dumped int a yaml
- (it's not the recommended stratgy--unless you have a specific reason for using this method, we recommend the following "Ring Optimized" notebook!)

In [ ]:
from pathlib import Path
from skymap_to_poly_coords import load_pickle_skymap, write_polygons_ra_dec

## Read in raw skymap

In [2]:
package_root = Path.home() / "skymap-to-poly-coords"

raw_skymaps_dir = package_root / "tests" / "data" / "raw_skymaps"
raw_skymap_path = raw_skymaps_dir / "skyMap_lsst_cells_v1_skymaps.pickle"

In [3]:
lsst_skymap = load_pickle_skymap(raw_skymap_path)
lsst_skymap

## Write Full Vertex file

See main "Convert skymaps" notebook for explanation on inner and outer polygons.

In [ ]:
converted_skymaps_dir = package_root / "converted_skymaps" / "full_vertex"

inner_poly_path = converted_skymaps_dir / "inner_polygons.yaml"
outer_poly_path = converted_skymaps_dir / "outer_polygons.yaml"

In [4]:
write_polygons_ra_dec(lsst_skymap, inner_poly_path, inner=True)
write_polygons_ra_dec(lsst_skymap, outer_poly_path, inner=False)

✅ Wrote 18938 tract polygons to /sdf/home/o/olynn/skymap-to-poly-coords/converted_skymaps/full_vertex/inner_polygons.yaml
✅ Wrote 18938 tract polygons to /sdf/home/o/olynn/skymap-to-poly-coords/converted_skymaps/full_vertex/outer_polygons.yaml


## Verify polygons against original skymap

In [ ]:
from skymap_to_poly_coords.test_utils import polys_are_equiv, get_poly_from_tract_id

def verify_polygons(fv_skymap, orig_skymap, inner=True):
    for tract_id in range(orig_skymap._numTracts):
        fv_poly = fv_skymap.get(tract_id)
        orig_poly = get_poly_from_tract_id(lsst_skymap, tract_id, inner=inner)

        if tract_id == 0 or tract_id == orig_skymap._numTracts - 1:
            # Skip first and last tract for now, but todo
            continue
        else:
            if not polys_are_equiv(fv_poly, orig_poly):
                raise ValueError(f"Tract {tract_id} polygons are not equivalent")
    print(f"All {'inner' if inner else 'outer'} tract polygons are equivalent.")

In [ ]:
print("Verifying inner tract polygons...")
verify_polygons